In [1]:
import plotly
import numpy as nd
import pandas as pd
import plotly.graph_objs as go
import functools
import allocate
plotly.offline.init_notebook_mode(connected=True)
%matplotlib inline

In [2]:
# Generate 30 days of orders using the Poisson distribution with the given rates
order_rates = [
    ("aaa", 400/30),
    ("bbb", 500/30),
    ("ccc", 500/30),
    ("xxx", 800/30),
    ("yyy", 800/30),
    ("zzz", 750/30)
]
order_days = 30
bookings = pd.concat(
    [
        pd.DataFrame({
            "BookingId": [part + "-" + str(i) for i in range(0,order_days)],
            "Priority": 100,
            "Part": part,
            "Quantity": pd.Series(nd.random.poisson(lam=lam, size=order_days)),
            "DueDate": pd.date_range(start="2018-01-15", periods=order_days)
        })
        for (part,lam) in order_rates
    ]
)
prev_parts = []
plotly.offline.iplot(
    go.Figure(
        data=
          [ go.Bar(
                x=bookings[bookings.Part==part]["DueDate"],
                y=bookings[bookings.Part==part]["Quantity"],
                name=part)
            for part in bookings.Part.unique()
          ],
        layout=go.Layout(barmode='stack')
))

In [3]:
results = allocate.allocate(
    bookings=bookings,
    prev_parts=prev_parts,
    flex_file="sample-flex.json",
    allocatecli="../allocatecli",
    plugin="../../orderlink/lib/pegboard/allocate/bin/Debug/netstandard2.0/BlackMaple.Pegboard.Allocate.dll")
allocate.print_result_summary(results)
plotly.offline.iplot(allocate.plot_simstat(allocate.simstat(results)))
plotly.offline.iplot(allocate.plot_simprod(results))

Simulation from 2016-11-05T12:00:00Z to 2016-11-06T13:00:00Z UTC
* aaa 
    Priority: 100
    Proc 1
        Path 0
            Completed 15
            Pallets 1,2,3,4
            Loads 1,2
            Stops Machine1,Machine2,Machine3,Machine4
            Unload 1,2
    Proc 2
        Path 0
            Pallets 1,2,3,4
            Loads 1,2
            Stops Machine1,Machine2,Machine3,Machine4
            Unload 1,2
* bbb 
    Priority: 100
    Proc 1
        Path 0
            Completed 20
            Pallets 1,2,3,4
            Loads 1,2
            Stops Machine1,Machine2,Machine3,Machine4
            Unload 1,2
    Proc 2
        Path 0
            Pallets 1,2,3,4
            Loads 1,2
            Stops Machine1,Machine2,Machine3,Machine4
            Unload 1,2
* ccc 
    Priority: 100
    Proc 1
        Path 0
            Completed 22
            Pallets 1,2,3,4
            Loads 1,2
            Stops Machine1,Machine2,Machine3,Machine4
            Unload 1,2
    Proc 2
        P

In [ ]:
allocate.download(results, "localhost:5000")